In [34]:
import plistlib
import mutagen
from pathlib import Path
from urllib.parse import unquote, urlparse
import subprocess
import os

In [14]:
os.chdir("..")

In [16]:
def get_playlists_by_name(name):
    return [pl for pl in playlists if pl["Name"] == name]


def get_parent(pl):
    if "Parent Persistent ID" not in pl.keys():
        return None
    else:
        parent = [item for item in playlists
        if item["Playlist Persistent ID"] == pl["Parent Persistent ID"]][0]
        return parent


def get_ancestors(pl):
    this_pl = pl
    ancestors = []
    while "Parent Persistent ID" in this_pl.keys():
        this_plp = get_parent(this_pl)
        ancestors.append(this_plp)
        this_pl = this_plp
    return ancestors


def playlist_tracks(playlist):
    track_ids = [item["Track ID"] for item in playlist["Playlist Items"]]
    # return [track for i, track in itl["Tracks"].items() if track["Track ID"] in track_ids]
    playlist_tracks = []
    for track_id in track_ids:
        playlist_tracks.extend([track for i, track in itl["Tracks"].items() if track["Track ID"] == track_id])
    return playlist_tracks

In [17]:
with open(Path("/Users/toph/Music/Library.xml").absolute(), "rb") as xml_file:
    itl = plistlib.load(xml_file)


## Transform data

### Vibes

In [18]:
playlists = itl["Playlists"]

In [51]:
vibes_folder = [pl for pl in playlists if pl["Name"] == "2. Vibes"][0]

In [52]:
music_vibes = [
    pl for pl in playlists
    if vibes_folder in get_ancestors(pl)
]

In [55]:
sql_vibes = []
for pl in music_vibes:
    vibe = {}
    vibe["vibe_id"] = vibe["playlist_id"] = pl["Playlist ID"]
    vibe["name"] = pl["Name"]
    vibe["parent_id"] = get_parent(pl)["Playlist ID"]
    vibe["type"] = "label" if "Folder" not in pl.keys() else "folder"
    sql_vibes.append(vibe)

In [57]:
sql_vibes[0:6]

[{'vibe_id': 163260,
  'playlist_id': 163260,
  'name': '1. Elements – Instruments, Beats, Composition, etc.',
  'parent_id': 162450,
  'type': 'folder'},
 {'vibe_id': 163654,
  'playlist_id': 163654,
  'name': '1. Instruments, Sounds',
  'parent_id': 163260,
  'type': 'folder'},
 {'vibe_id': 163739,
  'playlist_id': 163739,
  'name': 'Acoustic Parts',
  'parent_id': 163654,
  'type': 'label'},
 {'vibe_id': 163756,
  'playlist_id': 163756,
  'name': 'Beatless, Ambient',
  'parent_id': 163654,
  'type': 'label'},
 {'vibe_id': 163775,
  'playlist_id': 163775,
  'name': 'Beats Mostly',
  'parent_id': 163654,
  'type': 'label'},
 {'vibe_id': 163784,
  'playlist_id': 163784,
  'name': 'Beats Only',
  'parent_id': 163654,
  'type': 'label'}]

### Tracks

In [76]:
def try_key(dict, key, default):
    return dict[key] if key in dict.keys() else default

In [75]:
sql_tracks = []

for _, music_track in itl["Tracks"].items():
    track = {}
    track["track_id"] = music_track["Track ID"]
    track["name"] = music_track["Name"]
    track["artist"] = music_track["Artist"]
    track["tracknum"] = music_track["Track Number"] if "Track Number" in music_track.keys() else 0
    sql_tracks.append(track)


KeyError: 'Artist'

In [71]:
music_tracks[0]

{'Track ID': 15129,
 'Name': 'Music on My Teeth (feat. José Gonzalez)',
 'Artist': 'DJ Koze',
 'Album Artist': 'DJ Koze',
 'Composer': 'Stefan Kozalla & José González',
 'Album': 'Knock Knock',
 'Genre': 'Electronica',
 'Kind': 'Apple Music AAC audio file',
 'Size': 9076829,
 'Total Time': 256347,
 'Disc Number': 1,
 'Disc Count': 1,
 'Track Number': 5,
 'Track Count': 16,
 'Year': 2018,
 'Date Modified': datetime.datetime(2019, 8, 22, 16, 11, 45),
 'Date Added': datetime.datetime(2018, 12, 14, 21, 24, 27),
 'Bit Rate': 256,
 'Sample Rate': 44100,
 'Release Date': datetime.datetime(2018, 5, 4, 7, 0),
 'Rating': 80,
 'Album Rating': 80,
 'Album Rating Computed': True,
 'Loved': True,
 'Normalization': 1440,
 'Sort Album': 'Knock Knock',
 'Sort Artist': 'DJ Koze',
 'Sort Name': 'Music on My Teeth (feat. José Gonzalez)',
 'Persistent ID': '923F50CB14490388',
 'Track Type': 'File',
 'Protected': True,
 'Apple Music': True,
 'Location': 'file:///Users/toph/Music/iTunes/iTunes%20Media/Apple%

In [ ]:
music_tracks = [
    track_id = 
]

## Initialize Schema

In [43]:
subprocess.run("rm -f vibechecker.db".split(" "))
subprocess.run("sqlite3 -init init-db.sql vibechecker.db '.exit'".split(" "))

CompletedProcess(args=['sqlite3', '-init', 'init-db.sql', 'vibechecker.db', "'.exit'"], returncode=1)

## Insert stuff into tables!

In [58]:
import sqlite3

In [64]:
conn = sqlite3.connect("vibechecker.db")

In [60]:
cursor = conn.cursor()

In [61]:
cursor.executemany("""
    INSERT INTO vibes
        (vibe_id, playlist_id, name, parent_id, type)
    VALUES
        (:vibe_id, :playlist_id, :name, :parent_id, :type)""", sql_vibes)

In [62]:
conn.commit()